In [1]:
import numpy as np
from functools import lru_cache
from sklearn.cluster import AgglomerativeClustering
from sklearn.cluster import DBSCAN as sk_DBSCAN
import timeit
import pandas as pd
import time

In [2]:
class Cluster:
    def __init__(self, index, is_leaf=False, coordinates=None, n_points=1, feature_indexes=[], child_1=None, child_2=None):
        self.index = index
        self.coordinate = coordinates
        self.n_points = n_points
        self.feature_indexes = feature_indexes
        self.child_1 = child_1
        self.child_2 = child_2
        self.is_leaf = is_leaf

In [3]:
class HierarchyClust:
    def __init__(self, n_clusters, metric='euclidean', linkage='ward'):
        self.n_clusters = n_clusters
        self.metric = metric
        self.linkage = linkage
        self.clusters = {}
        self.cluster_creation_history = -1
        self.x = None

    @staticmethod
    @lru_cache
    def get_euclidean_distance(points_set):
        point_1, point_2 = points_set
        return np.linalg.norm(np.array(point_1) - np.array(point_2), axis=0)

    def get_wards_coefs(self, s, u, v):
        alpha_u = (s.n_points + u.n_points) / (s.n_points + (u.n_points + v.n_points))
        alpha_v = (s.n_points + v.n_points) / (s.n_points + (u.n_points + v.n_points))
        beta = -s.n_points / (s.n_points + (u.n_points + v.n_points))
        return alpha_u, alpha_v, beta

    @lru_cache
    def get_distance(self, cluster_set):
        cluster_1, cluster_2 = cluster_set
        if cluster_1.is_leaf and cluster_2.is_leaf:
            return self.get_euclidean_distance(frozenset([cluster_1.coordinate, cluster_2.coordinate]))
        elif not cluster_1.is_leaf or not cluster_2.is_leaf:
            s_cluster = cluster_2
            w_cluster = cluster_1
            if cluster_1.index < cluster_2.index:
                s_cluster = cluster_1
                w_cluster = cluster_2
            al_u, al_v, b = self.get_wards_coefs(s_cluster, w_cluster.child_1, w_cluster.child_2)
            return al_u * self.get_distance(frozenset([w_cluster.child_1, s_cluster])) + \
                al_v * self.get_distance(frozenset([w_cluster.child_2, s_cluster])) + \
                b * self.get_distance(frozenset([w_cluster.child_1, w_cluster.child_2]))

    def get_distance_matrix(self):
        current_clusters_idxs = self.clusters.keys()
        max_clustr_idx = max(current_clusters_idxs)
        distance_matrix = np.full((max_clustr_idx + 1, max_clustr_idx + 1), np.inf)
        for i in range(distance_matrix.shape[1]):
            for j in range(i + 1, distance_matrix.shape[1]):
                if i not in current_clusters_idxs or j not in current_clusters_idxs:
                    pass
                else:
                    distance_matrix[i, j] = self.get_distance(frozenset([self.clusters[i], self.clusters[j]]))
        return distance_matrix

    def fit(self, x):
        assert x.shape[0] > self.n_clusters, 'Objects number exceed clusters number'
        self.x = x
        for i in range(x.shape[0]):
            self.cluster_creation_history += 1
            self.clusters[self.cluster_creation_history] = Cluster(
                self.cluster_creation_history,
                coordinates=tuple(x[self.cluster_creation_history]),
                feature_indexes=[i],
                is_leaf=True)
        while len(self.clusters) > self.n_clusters:
            distance_matrix = self.get_distance_matrix()
            closest = np.argwhere(distance_matrix == np.min(distance_matrix))[0]
            self.cluster_creation_history += 1
            self.clusters[self.cluster_creation_history] = Cluster(
                self.cluster_creation_history,
                child_1=self.clusters[closest[0]],
                child_2=self.clusters[closest[-1]],
                n_points=self.clusters[closest[0]].n_points + self.clusters[closest[-1]].n_points,
                feature_indexes=self.clusters[closest[0]].feature_indexes + self.clusters[closest[-1]].feature_indexes
            )
            self.clusters.pop(closest[0])
            self.clusters.pop(closest[-1])

In [4]:
a = np.array([
    [4,   6],
    [2,   2],
    [7,   3],
    [2,   6],
    [7,   2],
    [2.8, 2],
    [1,   5]
])

In [5]:
#hc = HierarchyClust(2)
#hc.fit(a)
mine = 'hc = HierarchyClust(2);hc.fit(a)'

#sk = AgglomerativeClustering(n_clusters=2)
#sk.fit(a)
sks = 'sk = AgglomerativeClustering(n_clusters=2);sk.fit(a)'

In [6]:
print('MINE time:', timeit.timeit(mine, globals=globals(), number=1000))

MINE time: 0.2698823999999993


In [7]:
print('SKLEARN time: ', timeit.timeit(sks, globals=globals(), number=1000))

SKLEARN time:  0.2032216


**DBSCAN**

In [8]:
class DBSCAN:
    def __init__(self, eps, min_poins=3, metric='euclidean'):
        self.eps = eps
        self.min_poins = min_poins
        self.visited = None
        self.clusters = {-1: []}
        self.clusterized = None
        self.features = None

    @staticmethod
    def euclidean(x_1, x_2):
        return np.linalg.norm(x_1 - x_2, axis=0)

    def get_neighbors(self, point, point_idx):
        all_closest = np.where(np.linalg.norm(self.features[:] - point, axis=1) <= self.eps)
        return np.delete(all_closest, np.argwhere(all_closest == point_idx))

    def expand_cluster(self, neighbors):
        for each in neighbors:
            if self.visited[each]:
                continue
            self.visited[each] = True
            self.clusterized.append(each)
            return self.expand_cluster(self.get_neighbors(self.features[each], each))

    def fit(self, x):
        grey_zone = []
        self.features = x
        self.visited = np.full((x.shape[0],), False)
        for i in range(self.features.shape[0]):
            if self.visited[i]:
                continue
            self.visited[i] = True
            neighbors = self.get_neighbors(self.features[i], i)
            self.clusterized = [i]
            self.expand_cluster(neighbors)
            if len(self.clusterized) < self.min_poins:
                grey_zone += self.clusterized
            else:
                self.clusters[max(self.clusters) + 1] = self.clusterized
        self.clusters[-1] = grey_zone

In [9]:
a = np.array([
    [5,  3],
    [2,  6],
    [10, 6],
    [10, 7],
    [6,  5],
    [3,  6],
    [5,  4],
    [10, 5],
    [6,  4],
    [7,  5],
    [10, 2],
    [7,  4],
    [10, 4],
    [10, 3]
])

In [10]:
# db = DBSCAN(2)
# db.fit(a)
mine = 'db = DBSCAN(2);db.fit(a)'


# sk = sk_DBSCAN(eps=2)
# sk.fit(a)
sks = 'sk = sk_DBSCAN(eps=2);sk.fit(a)'

In [11]:
print('MINE time: ', timeit.timeit(mine, globals=globals(), number=10000))

MINE time:  6.086742300000001


In [12]:
print('SKLEARN time: ', timeit.timeit(sks, globals=globals(), number=1000))

SKLEARN time:  0.8856695000000006


In [13]:
timeit.timeit(sks, globals=globals(), number=10000)

11.116450799999999

In [14]:
house_df = pd.read_csv('kc_house_data.csv', sep=',')
house_df

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,...,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
0,7129300520,20141013T000000,221900.0,3,1.00,1180,5650,1.0,0,0,...,7,1180,0,1955,0,98178,47.5112,-122.257,1340,5650
1,6414100192,20141209T000000,538000.0,3,2.25,2570,7242,2.0,0,0,...,7,2170,400,1951,1991,98125,47.7210,-122.319,1690,7639
2,5631500400,20150225T000000,180000.0,2,1.00,770,10000,1.0,0,0,...,6,770,0,1933,0,98028,47.7379,-122.233,2720,8062
3,2487200875,20141209T000000,604000.0,4,3.00,1960,5000,1.0,0,0,...,7,1050,910,1965,0,98136,47.5208,-122.393,1360,5000
4,1954400510,20150218T000000,510000.0,3,2.00,1680,8080,1.0,0,0,...,8,1680,0,1987,0,98074,47.6168,-122.045,1800,7503
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21608,263000018,20140521T000000,360000.0,3,2.50,1530,1131,3.0,0,0,...,8,1530,0,2009,0,98103,47.6993,-122.346,1530,1509
21609,6600060120,20150223T000000,400000.0,4,2.50,2310,5813,2.0,0,0,...,8,2310,0,2014,0,98146,47.5107,-122.362,1830,7200
21610,1523300141,20140623T000000,402101.0,2,0.75,1020,1350,2.0,0,0,...,7,1020,0,2009,0,98144,47.5944,-122.299,1020,2007
21611,291310100,20150116T000000,400000.0,3,2.50,1600,2388,2.0,0,0,...,8,1600,0,2004,0,98027,47.5345,-122.069,1410,1287


In [15]:
start = time.time()
mine_sk = sk_DBSCAN()
mine_sk.fit(house_df[['lat', 'long']].drop_duplicates().to_numpy())
print(time.time() - start)

11.349819660186768


### start = time.time()
mine_db = DBSCAN(0.05)
mine_db.fit(house_df[['lat', 'long']].drop_duplicates().to_numpy())
print(time.time() - start)

In [18]:
%timeit for i in range(10 ** 7): pass

257 ms ± 4.64 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [19]:
10**12 * 4 / 2 ** 30

3725.290298461914